# Sports Statistics

Below you'll find statistics for the following sports:
- Ice Hockey
- Athletics
- Shooting
- Cross Country Skiing

(The four sports that Sweden got the most medals in)

In [2]:
import pandas as pd
import plotly_express as px

In [3]:
# Reading the athlete_events.csv file
athlete_events = pd.read_csv("data/athlete_events.csv")

In [9]:
# NOTE: Create a new DataFrame instance named df_icehockey using pandas.
# Filter the rows from the existing DataFrame athlete_events based on the condition 'Sport == "XX"'.
# The resulting DataFrame, df_icehockey, contains data specific to athletes and events in the sport "XX".
df_icehockey = pd.DataFrame(athlete_events.query('Sport == "Ice Hockey"'))
df_athletics = pd.DataFrame(athlete_events.query('Sport == "Athletics"'))
df_shooting = pd.DataFrame(athlete_events.query('Sport == "Shooting"'))
df_cross_county_skiing = pd.DataFrame(athlete_events.query('Sport == "Cross Country Skiing"'))

selected_sports = ['Ice Hockey', 'Athletics', 'Shooting', 'Cross Country Skiing']
df_all_selected_sports = pd.DataFrame(athlete_events.query('Sport in @selected_sports'))

# NOTE: Displaying the length of the df:s using display method
display(f"{len(df_icehockey)=}")
display(f"{len(df_athletics)=}")
display(f"{len(df_shooting)=}")
display(f"{len(df_cross_county_skiing)=}")
display(f"{len(df_all_selected_sports)=}")

'len(df_icehockey)=5516'

'len(df_athletics)=38624'

'len(df_shooting)=11448'

'len(df_cross_county_skiing)=9133'

'len(df_all_selected_sports)=64721'

#### Medaljfördelning mellan länder i sporterna

#### Åldersfördelning i sporterna

#### Skapa fler plots för att visualisera olika aspekter kring sporterna.